In [53]:
import simpy as sy
import statistics
import random



In [54]:
wait_times = []

In [55]:
class MovieHouse(object):
    def __init__(self, env, num_cashiers, num_ushers, num_servers):
        self.env = env
        self.cashier = sy.Resource(env, num_cashiers)
        self.usher = sy.Resource(env, num_ushers)
        self.server = sy.Resource(env, num_servers)
    #assume moviegoer completes purchase in 2 mins
    def purchase_tickets(self, moviegoer):
        yield self.env.timeout(random.randint(1,3))
    #assume usher checks tickets for 3 secs
    def check_tickets(self, moviegoer):
        yield self.env.timeout(3/60)
    # 5 min food sir!
    def sell_food(self,moviegoer):
        yield self.env.timeout(random.randint(1,5))



In [56]:
def go_to_movies(env, moviegoer, moviehouse):
    #moviegoer arrives at the moviehouse
    arrival_time = env.now

    with moviehouse.cashier.request() as request:
        yield request
        yield env.process(moviehouse.purchase_tickets(moviegoer))
    with moviehouse.usher.request() as request:
        yield request
        yield env.process(moviehouse.check_tickets(moviegoer))
    if random.choice([True, False]):
        with moviehouse.server.request() as request:
            yield request
            yield env.process(moviehouse.sell_food(moviegoer))
    wait_times.append(env.now - arrival_time)

In [57]:
def run_moviehouse(env, num_cashiers, num_ushers, num_servers):
    movieHouse = MovieHouse(env, num_cashiers, num_ushers, num_servers)

    #assume when moviehouse opens, there are already 3 customers waiting
    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, movieHouse))
    while True:
        #12 seconds
        yield env.timeout(12/60)
        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, movieHouse))

In [58]:
def get_average_wait_times(wait_times):
    average_wait = statistics.mean(wait_times)
    minutes, frac_min = divmod(average_wait, 1)
    seconds = frac_min * 60
    return round(minutes), round(seconds)


In [59]:
def get_user_input():
    num_cashiers = input("Number of cashiers: ")
    num_ushers = input("Number of ushers: ")
    num_servers = input("Number of servers: ")
    params = [num_cashiers, num_ushers, num_servers]
    if all(str(i).isdigit() for i in params): #check input if valid
        params =[int(x) for x in params]
    else:
        print(
            "Could no parse input. Simulation will use default values:"
            "\n1 cashier, 1 server, 1 usher."
        )
        params = [1,1,1]
    return params

In [60]:
def main():
    random.seed(42)
    num_cashiers, num_ushers, num_servers = get_user_input()

    env = sy.Environment()
    env.process(run_moviehouse(env, num_cashiers, num_ushers, num_servers))
    env.run(until=90)

    minutes, seconds = get_average_wait_times(wait_times)
    print("Running simulation.....",
          f"\nThe average wait time is {minutes} minutes and {seconds} seconds")

In [61]:
main()